In [6]:
import numpy as np
import cv2

template = cv2.imread('Template-1.png')
hsv_frame = cv2.cvtColor(template, cv2.COLOR_BGR2HSV)
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
avg_color_per_row = np.average(hsv_frame, axis=0)
avg_color_template = np.average(avg_color_per_row, axis=0)

Color1 = np.array([int(avg_color_template[0]),int(avg_color_template[1]+10),30])
Color2 = np.array([100,255,255])

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (0, 255, 255)
thickness = 3

IP_file = 'left_output-1.avi'
vid_file = cv2.VideoCapture(IP_file)

while True:
    (read_successful, frame) = vid_file.read()
    query_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if read_successful:
        sift = cv2.SIFT_create()
        bf = cv2.BFMatcher()
        template_kpts, template_desc = sift.detectAndCompute(template_gray, None)

        dst3 = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        hsv = cv2.cvtColor(dst3, cv2.COLOR_BGR2HSV)
        
        mask = cv2.inRange(hsv, Color1, Color2)
        
        kernel2 = np.ones((3,3),np.uint8)
        kernel3 = np.ones((4,4),np.uint8)
        kernel4 = np.ones((5,5),np.uint8)
        kernel5 = np.ones((7,7),np.uint8)
        
        opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel2)
        dilation = cv2.dilate(opening,kernel5,iterations = 2)
        
        erosion = cv2.erode(dilation,kernel4,iterations = 2)
        opening2 = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel2)
        erosion2 = cv2.erode(opening2,kernel5,iterations = 2)
        opening3 = cv2.morphologyEx(erosion2, cv2.MORPH_OPEN, kernel5)
        
        dilation2 = cv2.dilate(opening3,kernel3,iterations = 1)
        
        contours, hier = cv2.findContours(dilation2,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        w_max = frame.shape[1] / 2
        h_max = frame.shape[0] / 2
        
        for cnt in contours:
            Z = cv2.contourArea(cnt)
            if 1350<Z:
                (x,y,w,h) = cv2.boundingRect(cnt)
                
                x1 = x-30
                y1 = y-30-int(w*0.55)
                
                org = (x1+int(w*0.5),y)
                Cur_X = (org[0]-w_max)*15/w_max
                Cur_Y = (h_max - org[1])*15/h_max
                Cur_Z = (20000-Z)*15/20000
                
                str1 = 'X:' + "{:.1f}".format(Cur_X) + ' Y:' +"{:.1f}".format(Cur_Y) + ' Z:' +"{:.1f}".format(Cur_Z)
                
                
                
                cropped_image = query_gray[y1:y+h+30 , x1:x+w+30]
                query_kpts, query_desc = sift.detectAndCompute(cropped_image, None)

                matches = bf.knnMatch(template_desc, query_desc, k=2)
        
                good_matches = list()
                for m, n in matches :
                    if m.distance < 0.5*n.distance :
                        good_matches.append(m)

                if len(good_matches) > 7 :
                    src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
                    dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

                    H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 10.0)
                    h2, w2 = template.shape[:2]
                    
                    template_box = np.float32([[0, 0], [0, h2-1], [w2-1, h2-1], [w2-1, 0]]).reshape(-1,1,2)
                    transformed_box = cv2.perspectiveTransform(template_box, H)

                    detected_img = cv2.polylines(frame[y1:y+h+30 , x1:x+w+30], [np.int32(transformed_box)], True, (0,255,0), 3, cv2.LINE_AA)
                    frame[y1:y+h+30 , x1:x+w+30] = detected_img

                image = cv2.putText(frame,str1, org, font,fontScale, color, thickness, cv2.LINE_AA)    
    else:
        break
    cv2.imshow('Detect book',frame) 
    key = cv2.waitKey(1)
    if key == 27:
        break
vid_file.release()
cv2.destroyAllWindows()